In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -q datasets

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# reference: https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

In [ ]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.readers import InputExample
from datasets import IterableDataset

In [ ]:
full_data_exploded_sector_grp_after_manual = pd.read_excel('sectorwise_budget_text_extracted.xlsx')
full_data_exploded_sector_grp_after_manual

date       year budget_full_text_filename  \
0    1947-11-26  1947-1948              bs194748.txt   
1    1947-11-26  1947-1948              bs194748.txt   
2    1947-11-26  1947-1948              bs194748.txt   
3    1947-11-26  1947-1948              bs194748.txt   
4    1947-11-26  1947-1948              bs194748.txt   
...         ...        ...                       ...   
1666 2025-01-02  2025-2026             bs2025_26.txt   
1667 2025-01-02  2025-2026             bs2025_26.txt   
1668 2025-01-02  2025-2026             bs2025_26.txt   
1669 2025-01-02  2025-2026             bs2025_26.txt   
1670 2025-01-02  2025-2026             bs2025_26.txt   

                                      sector  \
0                        Aerospace & Defence   
1                             Agro Chemicals   
2                                      Banks   
3       Capital Goods - Electrical Equipment   
4     Capital Goods-Non Electrical Equipment   
...                                      ...   
1666           Readymade Garments/ Apparells   
1667                                  Realty   
1668                           Ship Building   
1669                                Shipping   
1670     Telecomm Equipment & Infra Services   

                                           text_segment  
0     ['Reconstitution of Armed Forces... future dev...  
1     ['Large scale importation of foodgrains at a c...  
2     ['Stringent restrictions on imports... restric...  
3     ['Need for rapid industrialisation... scope fo...  
4     ['Need for rapid industrialisation... scope fo...  
...                                                 ...  
1666  ['Focus Product Scheme for Footwear & Leather ...  
1667  ['Development of 50 tourist destinations with ...  
1668  ['Shipbuilding Financial Assistance Policy wil...  
1669  ['Shipbuilding Financial Assistance Policy wil...  
1670  ['Reduction in BCD on Carrier-Grade Ethernet S...  

[1671 rows x 5 columns]

In [ ]:
full_data_exploded_sector_grp_after_manual['date'].nunique()

97

In [ ]:
full_data_exploded_sector_grp_after_manual['text_segment'] = full_data_exploded_sector_grp_after_manual['text_segment'].apply(lambda x : eval(x)[:4])


In [ ]:
full_data_exploded_sector_grp_after_manual['sector'].nunique()

81

In [ ]:
full_data_exploded_sector_grp_after_manual['sector'].value_counts()

sector
Textiles                           72
Fertilizers                        64
Steel                              60
Financial Services                 57
Power Generation & Distribution    53
                                   ..
Computer Education                  2
Ferro Alloys                        2
Plywood Boards/Laminates            1
Dry cells                           1
Printing & Stationery               1
Name: count, Length: 81, dtype: int64

In [ ]:
full_data_exploded_sector_grp_after_manual['year_use'] = full_data_exploded_sector_grp_after_manual['year'].apply(lambda x : int(str(x)[:4]))

full_data_exploded_sector_grp_after_manual

date       year budget_full_text_filename  \
0    1947-11-26  1947-1948              bs194748.txt   
1    1947-11-26  1947-1948              bs194748.txt   
2    1947-11-26  1947-1948              bs194748.txt   
3    1947-11-26  1947-1948              bs194748.txt   
4    1947-11-26  1947-1948              bs194748.txt   
...         ...        ...                       ...   
1666 2025-01-02  2025-2026             bs2025_26.txt   
1667 2025-01-02  2025-2026             bs2025_26.txt   
1668 2025-01-02  2025-2026             bs2025_26.txt   
1669 2025-01-02  2025-2026             bs2025_26.txt   
1670 2025-01-02  2025-2026             bs2025_26.txt   

                                      sector  \
0                        Aerospace & Defence   
1                             Agro Chemicals   
2                                      Banks   
3       Capital Goods - Electrical Equipment   
4     Capital Goods-Non Electrical Equipment   
...                                      ...   
1666           Readymade Garments/ Apparells   
1667                                  Realty   
1668                           Ship Building   
1669                                Shipping   
1670     Telecomm Equipment & Infra Services   

                                           text_segment  year_use  
0     [Reconstitution of Armed Forces... future deve...      1947  
1     [Large scale importation of foodgrains at a co...      1947  
2     [Stringent restrictions on imports... restrict...      1947  
3     [Need for rapid industrialisation... scope for...      1947  
4     [Need for rapid industrialisation... scope for...      1947  
...                                                 ...       ...  
1666  [Focus Product Scheme for Footwear & Leather S...      2025  
1667  [Development of 50 tourist destinations with i...      2025  
1668  [Shipbuilding Financial Assistance Policy will...      2025  
1669  [Shipbuilding Financial Assistance Policy will...      2025  
1670  [Reduction in BCD on Carrier-Grade Ethernet Sw...      2025  

[1671 rows x 6 columns]

In [ ]:
yr1 = 2020
yr2 = 2024
train = full_data_exploded_sector_grp_after_manual[full_data_exploded_sector_grp_after_manual['year_use']<yr1]
valid = full_data_exploded_sector_grp_after_manual[(full_data_exploded_sector_grp_after_manual['year_use']>=yr1) & (full_data_exploded_sector_grp_after_manual['year_use']<yr2)]
test = full_data_exploded_sector_grp_after_manual[full_data_exploded_sector_grp_after_manual['year_use']>=yr2]
train.shape, valid.shape, test.shape

((1521, 6), (79, 6), (71, 6))

In [ ]:
full_data_exploded_sector_grp_after_manual.shape

(1671, 6)

In [ ]:
1521/1671

In [ ]:
79/1671

In [ ]:
71/1671

In [ ]:
train_ex = train[['sector', 'text_segment']].explode('text_segment').dropna()
train_ex['score'] = 1.0
valid_ex = valid[['sector', 'text_segment']].explode('text_segment').dropna()
valid_ex['score'] = 1.0
test_ex = test[['sector', 'text_segment']].explode('text_segment').dropna()
test_ex['score'] = 1.0

train_ex['text_segment'] = train_ex['text_segment'].astype(str)
valid_ex['text_segment'] = valid_ex['text_segment'].astype(str)
test_ex['text_segment'] = test_ex['text_segment'].astype(str)

train_ex = train_ex.reset_index(drop=True)
valid_ex = valid_ex.reset_index(drop=True)
test_ex = test_ex.reset_index(drop=True)

np.random.seed(0)

In [ ]:
sectors = train_ex['sector'].unique()

# Create a mapping from sector names to numerical labels
sector_to_label = {sector: i for i, sector in enumerate(sectors)}

In [ ]:
sector_to_label

{'Aerospace & Defence': 0,
 'Agro Chemicals': 1,
 'Banks': 2,
 'Capital Goods - Electrical Equipment': 3,
 'Capital Goods-Non Electrical Equipment': 4,
 'Cement': 5,
 'Construction': 6,
 'Edible Oil': 7,
 'Engineering': 8,
 'Fertilizers': 9,
 'Financial Services': 10,
 'Healthcare': 11,
 'Infrastructure Developers & Operators': 12,
 'Insurance': 13,
 'Logistics': 14,
 'Mining & Mineral products': 15,
 'Plantation & Plantation Products': 16,
 'Power Generation & Distribution': 17,
 'Power Infrastructure': 18,
 'Ship Building': 19,
 'Shipping': 20,
 'Steel': 21,
 'Textiles': 22,
 'Automobile': 23,
 'Finance': 24,
 'Tobacco Products': 25,
 'Air Transport Service': 26,
 'Alcoholic Beverages': 27,
 'Auto Ancillaries': 28,
 'Cables': 29,
 'FMCG': 30,
 'Plywood Boards/Laminates': 31,
 'Railways': 32,
 'Sugar': 33,
 'Tyres': 34,
 'Chemicals': 35,
 'Infrastructure Investment Trusts': 36,
 'Paper': 37,
 'Telecomm Equipment & Infra Services': 38,
 'Telecomm-Service': 39,
 'Trading': 40,
 'Marine 

In [ ]:
model_name = "nomic-ai/nomic-embed-text-v1.5"
train_batch_size = 16
num_epochs = 4
output_dir = (
    "."
)


model = SentenceTransformer(model_name, trust_remote_code=True)
train_loss = losses.CosineSimilarityLoss(model=model)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

# Without Fine-tuning Sentence Transformer Nomic

In [ ]:
sectors = ['search_document: ' + i for i in train_ex['sector'].unique()]
corpus_embeddings = model.encode(sectors)
corpus = train_ex['sector'].unique()
import torch
print(corpus)

top_k = len(corpus)



predictions = []
for query in test_ex_grp['text_segment']:
    query_embedding = model.encode('search_query: ' + query)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = model.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)
    predictions.append([query,[corpus[idx] for idx in indices],scores])



['Aerospace & Defence' 'Agro Chemicals' 'Banks'
 'Capital Goods - Electrical Equipment'
 'Capital Goods-Non Electrical Equipment' 'Cement' 'Construction'
 'Edible Oil' 'Engineering' 'Fertilizers' 'Financial Services'
 'Healthcare' 'Infrastructure Developers & Operators' 'Insurance'
 'Logistics' 'Mining & Mineral products'
 'Plantation & Plantation Products' 'Power Generation & Distribution'
 'Power Infrastructure' 'Ship Building' 'Shipping' 'Steel' 'Textiles'
 'Automobile' 'Finance' 'Tobacco Products' 'Air Transport Service'
 'Alcoholic Beverages' 'Auto Ancillaries' 'Cables' 'FMCG'
 'Plywood Boards/Laminates' 'Railways' 'Sugar' 'Tyres' 'Chemicals'
 'Infrastructure Investment Trusts' 'Paper'
 'Telecomm Equipment & Infra Services' 'Telecomm-Service' 'Trading'
 'Marine Port & Services' 'Non Ferrous Metals' 'Electronics'
 'Paints/Varnish' 'Refineries' 'Education' 'Oil Drill/Allied'
 'Pharmaceuticals' 'Readymade Garments/ Apparells' 'Consumer Durables'
 'E-Commerce/App based Aggregator' 'Ca

In [ ]:
test_ex_grp

text_segment               label  \
0   100% FDI allowed in the insurance sector with ...            [10, 13]   
1   A Maritime Development Fund with a corpus of ₹...            [12, 20]   
2   A corpus of ₹1 lakh crore will be established ...     [0, 43, 11, 65]   
3   A new scheme will be launched for strengthenin...         [0, 43, 73]   
4   Comprehensive development of Vishnupad Temple ...            [59, 63]   
5   Customs duty exemption on 36 lifesaving drugs ...            [11, 48]   
6   Customs duty exemption on Wet Blue Leather and...                [57]   
7   Customs duty reduction on Frozen Fish Paste (S...            [30, 41]   
8   Development of 50 tourist destinations with in...        [59, 74, 63]   
9   Development of indigenous technology for Advan...         [3, 17, 21]   
10  Financial support for setting up a network of ...            [14, 41]   
11  Five integrated aquaparks will be setup under ...        [30, 14, 41]   
12  Focus Product Scheme for Footwear & Leather Se...            [57, 49]   
13  For facilitating MSMEs to unlock their working...            [24, 14]   
14  I propose to reduce the BCD on mobile phone, m...        [51, 43, 73]   
15  Modified UDAN scheme to enhance regional conne...            [26, 12]   
16  National Manufacturing Mission to support Clea...         [23, 3, 17]   
17  National Mission on High Yielding Seeds will s...              [1, 9]   
18  New 109 high-yielding and climate-resilient va...             [1, 16]   
19  New airports, medical colleges and sports infr...         [6, 11, 12]   
20  Our Government will expand and strengthen the ...     [28, 23, 3, 18]   
21  Ownership, leasing and flagging reforms will b...        [41, 19, 20]   
22  Phased mandatory blending of compressed biogas...        [60, 67, 17]   
23  Policy for recovery of critical minerals from ...            [15, 42]   
24  Pradhan Mantri Kisan Sampada Yojana has benefi...          [1, 30, 9]   
25  Rationalization of TDS/TCS thresholds to reduc...             [2, 10]   
26  Reduction in BCD on Carrier-Grade Ethernet Swi...            [73, 38]   
27  Reduction in customs duty to 6% on gold and si...            [75, 64]   
28  Revamped Kisan Credit Card (KCC) loan limits f...          [1, 2, 30]   
29  Shipbuilding Financial Assistance Policy will ...            [19, 20]   
30  Tax rebate for incomes up to ₹12 lakh under th...                [30]   
31  The aviation sector has been galvanized with 5...         [0, 26, 12]   
32  The limit of Mudra loans will be enhanced to ₹...            [28, 24]   
33  Three major economic railway corridor programm...  [3, 5, 14, 41, 15]   
34  To provide relief to cancer patients, I propos...            [11, 48]   
35  Upgradation of Anganwadi centres under Saksham...            [30, 11]   
36  Viability gap funding will be provided for har...     [4, 15, 67, 17]   
37  We will set up a Critical Mineral Mission for ...            [15, 42]   

                                            label_use  
0   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...  
1   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
2   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...  
3   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
5   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...  
6   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
7   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
8   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
9   [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
10  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
11  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
12  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
13  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
14  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
15  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
16  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
17  [0, 1, 0, 0, 0, 0, 0, 0, 

In [ ]:
from sklearn.metrics import f1_score

th = 0
for kkdd in range(19):
  th = th +.05
  print("Threshold is: ", th)

  predictions_use = [[j for j,k in zip(i[1], i[2]) if k.item()>th] for i in predictions]
  all_predictions_prior = [[sector_to_label[j] for j in i] for i in predictions_use]
  all_predictions = []
  for i in all_predictions_prior:
   all_predictions.append([1 if ik in i else 0 for ik in range(len(sector_to_label))])

  # Calculate the F1 score
  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='macro')
  print("Macro: ", f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='micro')
  print("Micro: ",f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='weighted')
  print('Weighted', f1)
  print("\n\n --------------- \n\n")


Threshold is:  0.05
Macro:  0.05722589280917888
Micro:  0.059880239520958084
Weighted 0.14588107298260447


 --------------- 


Threshold is:  0.1
Macro:  0.05722589280917888
Micro:  0.059880239520958084
Weighted 0.14588107298260447


 --------------- 


Threshold is:  0.15000000000000002
Macro:  0.05722589280917888
Micro:  0.059880239520958084
Weighted 0.14588107298260447


 --------------- 


Threshold is:  0.2
Macro:  0.05722589280917888
Micro:  0.059880239520958084
Weighted 0.14588107298260447


 --------------- 


Threshold is:  0.25
Macro:  0.05722589280917888
Micro:  0.059880239520958084
Weighted 0.14588107298260447


 --------------- 


Threshold is:  0.3
Macro:  0.05808052419329936
Micro:  0.060664730558244596
Weighted 0.1472665340937755


 --------------- 


Threshold is:  0.35
Macro:  0.06821700233765361
Micro:  0.07075471698113207
Weighted 0.1699973785550553


 --------------- 


Threshold is:  0.39999999999999997
Macro:  0.08726550586886554
Micro:  0.09149484536082474
Weig

# Adding negative Samples and Fine-tuning Sentence Transformers

In [ ]:
all_sectors = list(train_ex['sector'].unique())

In [ ]:
len(all_sectors)

81

In [ ]:
train_ex.head()

sector  \
0                   Aerospace & Defence   
1                        Agro Chemicals   
2                                 Banks   
3  Capital Goods - Electrical Equipment   
4  Capital Goods - Electrical Equipment   

                                        text_segment  score  
0  Reconstitution of Armed Forces... future devel...    1.0  
1  Large scale importation of foodgrains at a cos...    1.0  
2  Stringent restrictions on imports... restricti...    1.0  
3  Need for rapid industrialisation... scope for ...    1.0  
4  £65 million current account for imports of cap...    1.0

In [ ]:
train_ex.shape

(1930, 3)

In [ ]:
for text_segment in train_ex['text_segment']:
  corresponding_sectors = train_ex[train_ex['text_segment']==text_segment]['sector'].unique()
  other_sectors = list(set(all_sectors) - set(corresponding_sectors))
  random_other_sector = np.random.choice(other_sectors)
  new_row = {'sector': random_other_sector, 'text_segment': text_segment, 'score': 0.0}
  train_ex = pd.concat([train_ex, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
train_ex.shape

(3860, 4)

In [ ]:
train_ex

sector  \
0                      Aerospace & Defence   
1                           Agro Chemicals   
2                                    Banks   
3     Capital Goods - Electrical Equipment   
4     Capital Goods - Electrical Equipment   
...                                    ...   
3855                                 Sugar   
3856                              Bearings   
3857         Readymade Garments/ Apparells   
3858                         Miscellaneous   
3859                            Edible Oil   

                                           text_segment  score  
0     Reconstitution of Armed Forces... future devel...    1.0  
1     Large scale importation of foodgrains at a cos...    1.0  
2     Stringent restrictions on imports... restricti...    1.0  
3     Need for rapid industrialisation... scope for ...    1.0  
4     £65 million current account for imports of cap...    1.0  
...                                                 ...    ...  
3855  100% FDI permitted for insurance intermediarie...    0.0  
3856  Customs duty increased on auto parts, CCTV cam...    0.0  
3857  Blueprint to develop gas grids, water grids an...    0.0  
3858  Proposal to permit FIIs/FPIs to subscribe to l...    0.0  
3859  Additional income tax deduction of ₹1.5 lakh o...    0.0  

[3860 rows x 3 columns]

In [ ]:
train_ex['score'].value_counts()

score
1.0    1930
0.0    1930
Name: count, dtype: int64

In [ ]:
train_ex

sector  \
0                      Aerospace & Defence   
1                           Agro Chemicals   
2                                    Banks   
3     Capital Goods - Electrical Equipment   
4     Capital Goods - Electrical Equipment   
...                                    ...   
3855                      Oil Drill/Allied   
3856                              Shipping   
3857                           Fertilizers   
3858                                Cables   
3859                            Refineries   

                                           text_segment  score  label  
0     Reconstitution of Armed Forces... future devel...    1.0    0.0  
1     Large scale importation of foodgrains at a cos...    1.0    1.0  
2     Stringent restrictions on imports... restricti...    1.0    2.0  
3     Need for rapid industrialisation... scope for ...    1.0    3.0  
4     £65 million current account for imports of cap...    1.0    3.0  
...                                                 ...    ...    ...  
3855  100% FDI permitted for insurance intermediarie...    0.0    NaN  
3856  Customs duty increased on auto parts, CCTV cam...    0.0    NaN  
3857  Blueprint to develop gas grids, water grids an...    0.0    NaN  
3858  Proposal to permit FIIs/FPIs to subscribe to l...    0.0    NaN  
3859  Additional income tax deduction of ₹1.5 lakh o...    0.0    NaN  

[3860 rows x 4 columns]

In [ ]:
dev_evaluator = EmbeddingSimilarityEvaluator(
    valid_ex['text_segment'],
    valid_ex['sector'],
    valid_ex['score'],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

test_evaluator = EmbeddingSimilarityEvaluator(
    test_ex['text_segment'],
    test_ex['sector'],
    test_ex['score'],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

from datasets import Dataset
train_dataset = Dataset.from_pandas(train_ex[["sector",	"text_segment",	"score"]])
eval_dataset = Dataset.from_pandas(valid_ex[["sector",	"text_segment",	"score"]])
test_dataset = Dataset.from_pandas(test_ex[["sector",	"text_segment",	"score"]])

from torch.utils.data import IterableDataset
import os
os.environ["WANDB_DISABLED"] = "true"

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=output_dir,
    # Optional training parameters:
    num_train_epochs=num_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=train_batch_size,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="sts",  # Will be used in W&B if `wandb` is installed,


)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
)

trainer.train()

test_evaluator(model)

sectors = ['search_document: ' + i for i in train_ex['sector'].unique()]
corpus_embeddings = model.encode(sectors)
corpus = train_ex['sector'].unique()
import torch
print(corpus)

top_k = len(corpus)
predictions = []
for query in test_ex_grp['text_segment']:
    query_embedding = model.encode('search_query: ' + query)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = model.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)
    predictions.append([query,[corpus[idx] for idx in indices],scores])
    # for score, idx in zip(scores, indices):
    #     print(corpus[idx], f"(Score: {score:.4f})")
    #     predictions.append([query,corpus[idx],f"(Score: {score:.4f})"])


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


['Aerospace & Defence' 'Agro Chemicals' 'Banks'
 'Capital Goods - Electrical Equipment'
 'Capital Goods-Non Electrical Equipment' 'Cement' 'Construction'
 'Edible Oil' 'Engineering' 'Fertilizers' 'Financial Services'
 'Healthcare' 'Infrastructure Developers & Operators' 'Insurance'
 'Logistics' 'Mining & Mineral products'
 'Plantation & Plantation Products' 'Power Generation & Distribution'
 'Power Infrastructure' 'Ship Building' 'Shipping' 'Steel' 'Textiles'
 'Automobile' 'Finance' 'Tobacco Products' 'Air Transport Service'
 'Alcoholic Beverages' 'Auto Ancillaries' 'Cables' 'FMCG'
 'Plywood Boards/Laminates' 'Railways' 'Sugar' 'Tyres' 'Chemicals'
 'Infrastructure Investment Trusts' 'Paper'
 'Telecomm Equipment & Infra Services' 'Telecomm-Service' 'Trading'
 'Marine Port & Services' 'Non Ferrous Metals' 'Electronics'
 'Paints/Varnish' 'Refineries' 'Education' 'Oil Drill/Allied'
 'Pharmaceuticals' 'Readymade Garments/ Apparells' 'Consumer Durables'
 'E-Commerce/App based Aggregator' 'Ca

In [ ]:
from sklearn.metrics import f1_score

th = 0
for kkdd in range(19):
  th = th +.05
  print("Threshold is: ", th)

  predictions_use = [[j for j,k in zip(i[1], i[2]) if k.item()>th] for i in predictions]
  all_predictions_prior = [[sector_to_label[j] for j in i] for i in predictions_use]
  all_predictions = []
  for i in all_predictions_prior:
   all_predictions.append([1 if ik in i else 0 for ik in range(len(sector_to_label))])

  # Calculate the F1 score
  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='macro')
  print("Macro: ", f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='micro')
  print("Micro: ",f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='weighted')
  print('Weighted', f1)
  print("\n\n --------------- \n\n")


Threshold is:  0.05
Macro:  0.07644650244828008
Micro:  0.09090909090909091
Weighted 0.18960663868097843


 --------------- 


Threshold is:  0.1
Macro:  0.09562540366916622
Micro:  0.11860174781523096
Weighted 0.23110995778117185


 --------------- 


Threshold is:  0.15000000000000002
Macro:  0.120527993497451
Micro:  0.15173527037933818
Weighted 0.28359338683393187


 --------------- 


Threshold is:  0.2
Macro:  0.14933702346872124
Micro:  0.18949536560247168
Weighted 0.3415025101678757


 --------------- 


Threshold is:  0.25
Macro:  0.17382276450951265
Micro:  0.23393316195372751
Weighted 0.39142425343609893


 --------------- 


Threshold is:  0.3
Macro:  0.20506605760303728
Micro:  0.2843601895734597
Weighted 0.45127406052206437


 --------------- 


Threshold is:  0.35
Macro:  0.23728678771105097
Micro:  0.3449612403100775
Weighted 0.5251276807851682


 --------------- 


Threshold is:  0.39999999999999997
Macro:  0.2668564893510427
Micro:  0.40274599542334094
Weighted 0.5749

# Encoder based models for classification

## BERT

In [ ]:
train_ex = train[['sector', 'text_segment']].explode('text_segment').dropna()
train_ex['score'] = 1.0
valid_ex = valid[['sector', 'text_segment']].explode('text_segment').dropna()
valid_ex['score'] = 1.0
test_ex = test[['sector', 'text_segment']].explode('text_segment').dropna()
test_ex['score'] = 1.0

train_ex['text_segment'] = train_ex['text_segment'].astype(str)
valid_ex['text_segment'] = valid_ex['text_segment'].astype(str)
test_ex['text_segment'] = test_ex['text_segment'].astype(str)

train_ex = train_ex.reset_index(drop=True)
valid_ex = valid_ex.reset_index(drop=True)
test_ex = test_ex.reset_index(drop=True)

np.random.seed(0)

train_ex.shape, valid_ex.shape, test_ex.shape

((1930, 3), (113, 3), (95, 3))

In [ ]:
sectors = train_ex['sector'].unique()

# Create a mapping from sector names to numerical labels
sector_to_label = {sector: i for i, sector in enumerate(sectors)}

# Apply the mapping to the label column
train_ex['label'] = train_ex['sector'].map(sector_to_label)
valid_ex['label'] = valid_ex['sector'].map(sector_to_label)
test_ex['label'] = test_ex['sector'].map(sector_to_label)

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [ ]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(sectors))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256)


train_dataset = Dataset.from_pandas(train_ex[['text_segment', 'label']].rename(columns = {'text_segment' : 'text'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid_ex[['text_segment', 'label']].rename(columns = {'text_segment' : 'text'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test_ex[['text_segment', 'label']].rename(columns = {'text_segment' : 'text'})).map(tokenize_function, batched=True)

Map:   0%|          | 0/1930 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1930
})

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 5. Define metrics
metric = evaluate.load("f1")

#from sklearn.metrics import f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')

# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,  # Change to your validation set
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 7. Train and evaluate
print("Starting training...")
trainer.train()

print("Evaluating model...")
results = trainer.evaluate()
print(f"Final evaluation results: {results}")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting training...


Evaluating model...


Final evaluation results: {'eval_loss': 3.0499818325042725, 'eval_f1': 0.27011157909127714, 'eval_runtime': 0.5383, 'eval_samples_per_second': 209.927, 'eval_steps_per_second': 14.862, 'epoch': 3.0}


In [ ]:
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test evaluation results: {test_results}")

Test evaluation results: {'eval_loss': 3.199439525604248, 'eval_f1': 0.19900535291117008, 'eval_runtime': 0.5379, 'eval_samples_per_second': 176.623, 'eval_steps_per_second': 11.155, 'epoch': 3.0}


In [ ]:
def predict_class(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class (emotion)
    # predicted_class = torch.argmax(outputs.logits, dim=1).item()

    # Get the confidence score (probability)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)

    # confidence = probabilities[0][predicted_class].item()

    # Get the emotion label
    # emotion_label = dataset["train"].features["label"].names[predicted_class]

    return probabilities



In [ ]:
test_ex

sector  \
0                   Aerospace & Defence   
1                   Aerospace & Defence   
2                   Aerospace & Defence   
3                        Agro Chemicals   
4                 Air Transport Service   
..                                  ...   
90                               Realty   
91                        Ship Building   
92                             Shipping   
93                             Shipping   
94  Telecomm Equipment & Infra Services   

                                         text_segment  score  label  
0   A new scheme will be launched for strengthenin...    1.0      0  
1   The aviation sector has been galvanized with 5...    1.0      0  
2   A corpus of ₹1 lakh crore will be established ...    1.0      0  
3   Pradhan Mantri Kisan Sampada Yojana has benefi...    1.0      1  
4   The aviation sector has been galvanized with 5...    1.0     26  
..                                                ...    ...    ...  
90  Development of 50 tourist destinations with in...    1.0     63  
91  Shipbuilding Financial Assistance Policy will ...    1.0     19  
92  Shipbuilding Financial Assistance Policy will ...    1.0     20  
93  A Maritime Development Fund with a corpus of ₹...    1.0     20  
94  Reduction in BCD on Carrier-Grade Ethernet Swi...    1.0     38  

[95 rows x 4 columns]

In [ ]:
test_ex_grp = test_ex.groupby('text_segment')['label'].apply(list).reset_index()
test_ex_grp['label_use'] = test_ex_grp['label'].apply(lambda x: [1 if i in x else 0 for i in range(len(sector_to_label))])
test_ex_grp

text_segment               label  \
0   100% FDI allowed in the insurance sector with ...            [10, 13]   
1   A Maritime Development Fund with a corpus of ₹...            [12, 20]   
2   A corpus of ₹1 lakh crore will be established ...     [0, 43, 11, 65]   
3   A new scheme will be launched for strengthenin...         [0, 43, 73]   
4   Comprehensive development of Vishnupad Temple ...            [59, 63]   
5   Customs duty exemption on 36 lifesaving drugs ...            [11, 48]   
6   Customs duty exemption on Wet Blue Leather and...                [57]   
7   Customs duty reduction on Frozen Fish Paste (S...            [30, 41]   
8   Development of 50 tourist destinations with in...        [59, 74, 63]   
9   Development of indigenous technology for Advan...         [3, 17, 21]   
10  Financial support for setting up a network of ...            [14, 41]   
11  Five integrated aquaparks will be setup under ...        [30, 14, 41]   
12  Focus Product Scheme for Footwear & Leather Se...            [57, 49]   
13  For facilitating MSMEs to unlock their working...            [24, 14]   
14  I propose to reduce the BCD on mobile phone, m...        [51, 43, 73]   
15  Modified UDAN scheme to enhance regional conne...            [26, 12]   
16  National Manufacturing Mission to support Clea...         [23, 3, 17]   
17  National Mission on High Yielding Seeds will s...              [1, 9]   
18  New 109 high-yielding and climate-resilient va...             [1, 16]   
19  New airports, medical colleges and sports infr...         [6, 11, 12]   
20  Our Government will expand and strengthen the ...     [28, 23, 3, 18]   
21  Ownership, leasing and flagging reforms will b...        [41, 19, 20]   
22  Phased mandatory blending of compressed biogas...        [60, 67, 17]   
23  Policy for recovery of critical minerals from ...            [15, 42]   
24  Pradhan Mantri Kisan Sampada Yojana has benefi...          [1, 30, 9]   
25  Rationalization of TDS/TCS thresholds to reduc...             [2, 10]   
26  Reduction in BCD on Carrier-Grade Ethernet Swi...            [73, 38]   
27  Reduction in customs duty to 6% on gold and si...            [75, 64]   
28  Revamped Kisan Credit Card (KCC) loan limits f...          [1, 2, 30]   
29  Shipbuilding Financial Assistance Policy will ...            [19, 20]   
30  Tax rebate for incomes up to ₹12 lakh under th...                [30]   
31  The aviation sector has been galvanized with 5...         [0, 26, 12]   
32  The limit of Mudra loans will be enhanced to ₹...            [28, 24]   
33  Three major economic railway corridor programm...  [3, 5, 14, 41, 15]   
34  To provide relief to cancer patients, I propos...            [11, 48]   
35  Upgradation of Anganwadi centres under Saksham...            [30, 11]   
36  Viability gap funding will be provided for har...     [4, 15, 67, 17]   
37  We will set up a Critical Mineral Mission for ...            [15, 42]   

                                            label_use  
0   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...  
1   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
2   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...  
3   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
5   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...  
6   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
7   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
8   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
9   [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
10  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
11  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
12  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
13  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...  
14  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
15  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...  
16  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
17  [0, 1, 0, 0, 0, 0, 0, 0, 

In [ ]:
# Example usage
text_to_classify = test_ex['text_segment'][0]
probabilities = predict_class(text_to_classify)


In [ ]:
all_predictions = []
th = .01
for i in range(test_ex_grp.shape[0]):
  text_to_classify = test_ex_grp['text_segment'][i]
  probabilities = predict_class(text_to_classify)
  predicted_classes = [1 if probabilities[0][i].item() > th else 0 for i in range(len(probabilities[0])) ]
  all_predictions.append(predicted_classes)

  f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='weighted')

In [ ]:
f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='weighted')

0.2652698895886139

In [ ]:
th = 0
for kkdd in range(19):
  th = th +.05
  print("Threshold is: ", th)
  all_predictions = []
  for i in range(test_ex_grp.shape[0]):
    text_to_classify = test_ex_grp['text_segment'][i]
    probabilities = predict_class(text_to_classify)
    predicted_classes = [1 if probabilities[0][i].item() > th else 0 for i in range(len(probabilities[0])) ]
    all_predictions.append(predicted_classes)
  # Calculate the F1 score
  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='macro')
  print("Macro: ", f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='micro')
  print("Micro: ",f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='weighted')
  print('Weighted', f1)
  print("\n\n --------------- \n\n")

Threshold is:  0.05
Macro:  0.17927086630790334
Micro:  0.48863636363636365
Weighted 0.42462438146648673


 --------------- 


Threshold is:  0.1
Macro:  0.1069077013521458
Micro:  0.3125
Weighted 0.2319799498746867


 --------------- 


Threshold is:  0.15000000000000002
Macro:  0.044444444444444446
Micro:  0.16216216216216217
Weighted 0.11368421052631582


 --------------- 


Threshold is:  0.2
Macro:  0.024279835390946504
Micro:  0.09615384615384616
Weighted 0.06385964912280702


 --------------- 


Threshold is:  0.25
Macro:  0.014814814814814817
Micro:  0.06
Weighted 0.042105263157894736


 --------------- 


Threshold is:  0.3
Macro:  0.014814814814814817
Micro:  0.06060606060606061
Weighted 0.042105263157894736


 --------------- 


Threshold is:  0.35
Macro:  0.014814814814814817
Micro:  0.061224489795918366
Weighted 0.042105263157894736


 --------------- 


Threshold is:  0.39999999999999997
Macro:  0.009876543209876543
Micro:  0.041237113402061855
Weighted 0.0252631578947368

## RoBERTa

In [ ]:
# Load tokenizer and model
model_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(sectors))

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256)

# # Tokenize datasets
# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# # Prepare datasets
# train_dataset = tokenized_datasets["train"].shuffle(seed=42)
# eval_dataset = tokenized_datasets["validation"]


train_dataset = Dataset.from_pandas(train_ex[['text_segment', 'label']].rename(columns = {'text_segment' : 'text'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid_ex[['text_segment', 'label']].rename(columns = {'text_segment' : 'text'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test_ex[['text_segment', 'label']].rename(columns = {'text_segment' : 'text'})).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# 5. Define metrics
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')

# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,  # Change to your validation set
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 7. Train and evaluate
print("Starting training...")
trainer.train()

print("Evaluating model...")
results = trainer.evaluate()
print(f"Final evaluation results: {results}")

test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test evaluation results: {test_results}")

def predict_class(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class (emotion)
    # predicted_class = torch.argmax(outputs.logits, dim=1).item()

    # Get the confidence score (probability)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)

    # confidence = probabilities[0][predicted_class].item()

    # Get the emotion label
    # emotion_label = dataset["train"].features["label"].names[predicted_class]

    return probabilities

test_ex_grp = test_ex.groupby('text_segment')['label'].apply(list).reset_index()
test_ex_grp['label_use'] = test_ex_grp['label'].apply(lambda x: [1 if i in x else 0 for i in range(len(sector_to_label))])

th = 0
for kkdd in range(19):
  th = th +.05
  print("Threshold is: ", th)
  all_predictions = []
  for i in range(test_ex_grp.shape[0]):
    text_to_classify = test_ex_grp['text_segment'][i]
    probabilities = predict_class(text_to_classify)
    predicted_classes = [1 if probabilities[0][i].item() > th else 0 for i in range(len(probabilities[0])) ]
    all_predictions.append(predicted_classes)
  # Calculate the F1 score
  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='macro')
  print("Macro: ", f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='micro')
  print("Micro: ",f1)

  f1 = f1_score(np.array(test_ex_grp['label_use'].to_list()), np.array(all_predictions), average='weighted')
  print('Weighted', f1)
  print("\n\n --------------- \n\n")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting training...


Evaluating model...


Final evaluation results: {'eval_loss': 3.3041775226593018, 'eval_f1': 0.1649484170176518, 'eval_runtime': 0.4716, 'eval_samples_per_second': 239.601, 'eval_steps_per_second': 16.963, 'epoch': 3.0}
Test evaluation results: {'eval_loss': 3.603489398956299, 'eval_f1': 0.09318921659153043, 'eval_runtime': 0.3251, 'eval_samples_per_second': 292.183, 'eval_steps_per_second': 18.454, 'epoch': 3.0}
Threshold is:  0.05
Macro:  0.07538702723887909
Micro:  0.273972602739726
Weighted 0.19254803675856305


 --------------- 


Threshold is:  0.1
Macro:  0.037037037037037035
Micro:  0.15517241379310345
Weighted 0.09473684210526316


 --------------- 


Threshold is:  0.15000000000000002
Macro:  0.02592592592592593
Micro:  0.09523809523809523
Weighted 0.05263157894736842


 --------------- 


Threshold is:  0.2
Macro:  0.009876543209876543
Micro:  0.04081632653061224
Weighted 0.025263157894736845


 --------------- 


Threshold is:  0.25
Macro:  0.009876543209876543
Micro:  0.04081632653061224
Weight